## Data retrieval and cleaning

In [1]:
from ift6758.data.acquisition import NHLGameData

data_path_raw = './../../ift6758/data/json_raw/'
nhl_games_data = NHLGameData(data_path_raw)
for year in range(2016,2021):
    nhl_games_data.fetch_season(year)



Loading from cache file ./../../ift6758/data/json_raw/2016/2016-regular.pkl
Found 1230 regular games for season 2016-2017
Loading from cache file ./../../ift6758/data/json_raw/2016/2016-playoff.pkl
Found 102 playoff games for season 2016-2017
Loading from cache file ./../../ift6758/data/json_raw/2017/2017-regular.pkl
Found 1271 regular games for season 2017-2018
Loading from cache file ./../../ift6758/data/json_raw/2017/2017-playoff.pkl
Found 105 playoff games for season 2017-2018
Loading from cache file ./../../ift6758/data/json_raw/2018/2018-regular.pkl
Found 1271 regular games for season 2018-2019
Loading from cache file ./../../ift6758/data/json_raw/2018/2018-playoff.pkl
Found 105 playoff games for season 2018-2019
Loading from cache file ./../../ift6758/data/json_raw/2019/2019-regular.pkl
Found 1271 regular games for season 2019-2020
Loading from cache file ./../../ift6758/data/json_raw/2019/2019-playoff.pkl
Found 105 playoff games for season 2019-2020
Loading from cache file ./..

In [2]:
from ift6758.data.cleaning import DataCleaner

data_path_clean = './../../ift6758/data/json_clean/'
data_cleaner = DataCleaner(data_raw=nhl_games_data, data_path_clean=data_path_clean)
for year in range(2016,2021):
    data_cleaner.clean_season(year, keepPreviousEventInfo=True, includePowerPlay=False)


In [3]:
data_2018 = data_cleaner.get_cleaned_data(2018)

In [4]:
data_2018.head()

,game_id,period,period_time,type,team,x,y,shooter,goalie,shot_type,empty_net,strength,opposite_team_side,prev_type,prev_x,prev_y,time_since_prev,distance_from_prev
0,2018020001,1,29,SHOT,Montréal Canadiens,78.0,-19.0,Artturi Lehkonen,Frederik Andersen,Backhand,False,None,right,FACEOFF,0.0,0.0,29.0,80.28
1,2018020001,1,49,SHOT,Toronto Maple Leafs,-37.0,-10.0,Morgan Rielly,Carey Price,Snap Shot,False,None,left,SHOT,78.0,-19.0,20.0,115.35
2,2018020001,1,60,SHOT,Montréal Canadiens,47.0,-23.0,Brendan Gallagher,Frederik Andersen,Wrist Shot,False,None,right,SHOT,-37.0,-10.0,11.0,85.00
3,2018020001,1,234,SHOT,Montréal Canadiens,53.0,14.0,Brendan Gallagher,Frederik Andersen,Snap Shot,False,None,right,MISSED_SHOT,73.0,22.0,5.0,21.54
4,2018020001,1,246,SHOT,Montréal Canadiens,77.0,-2.0,Mike Reilly,Frederik Andersen,Snap Shot,False,None,right,MISSED_SHOT,81.0,-4.0,5.0,4.47


In [5]:
data_2018.isna().sum()

game_id                   0
period                    0
period_time               0
type                      0
team                      0
x                         0
y                         0
shooter                   0
goalie                  420
shot_type                 0
empty_net                 0
strength              75063
opposite_team_side        0
prev_type                 0
prev_x                    0
prev_y                    0
time_since_prev           0
distance_from_prev        0
dtype: int64

## Feature engineering

In [6]:
from ift6758.features import FeatureEng
data_path_clean = './../../ift6758/data/json_clean/'
w = FeatureEng(data_path_clean)

In [7]:
df = w.features_2(2016,2020)
df.sample(10)

,game_id,period,game_seconds,x,y,shot_type,empty_net,strength,prev_type,prev_x,...,time_since_prev,distance_from_prev,distance_goal,prev_distance_goal,angle_shot,prev_angle_shot,bounce,angle_change,speed,is_goal
116963,2017020674,1,1341,-77.0,-6.0,Wrist Shot,False,None,BLOCKED_SHOT,-79.0,...,1.0,19.10,14.32,27.31,-24.77,-66.26,True,41.49,19.10,0
232918,2019020015,2,3430,34.0,-23.0,Wrist Shot,False,None,FACEOFF,69.0,...,172.0,57.01,60.54,30.41,-22.33,46.34,False,0.00,0.33,0
224504,2018021150,3,3850,39.0,29.0,Snap Shot,False,None,HIT,61.0,...,102.0,38.01,58.67,29.07,29.62,-3.95,False,0.00,0.37,0
31685,2016020522,3,3672,68.0,1.0,Wrist Shot,False,None,BLOCKED_SHOT,64.0,...,6.0,5.00,22.02,26.08,2.60,-4.40,True,7.00,0.83,0
193368,2018020636,3,4654,18.0,15.0,Wrist Shot,False,None,HIT,99.0,...,33.0,86.03,73.55,16.64,11.77,-57.28,False,0.00,2.61,0
190036,2018020584,2,3004,66.0,-6.0,Deflected,False,None,BLOCKED_SHOT,-58.0,...,10.0,124.20,24.74,148.57,-14.04,-5.02,True,-9.02,12.42,0
241293,2019020152,2,2685,-53.0,-17.0,Wrist Shot,False,None,FACEOFF,0.0,...,28.0,55.66,40.72,90.00,-24.68,0.00,False,0.00,1.99,0
77090,2017020032,3,4101,72.0,1.0,Wrist Shot,False,None,FACEOFF,20.0,...,72.0,56.86,18.03,73.38,3.18,-17.45,False,0.00,0.79,0
56266,2016020925,1,1835,-66.0,16.0,Snap Shot,False,Power Play,BLOCKED_SHOT,-79.0,...,2.0,22.20,28.84,11.18,33.70,-10.31,True,44.01,11.10,1
204372,2018020822,3,4524,-84.0,-9.0,Wrist Shot,False,None,FACEOFF,-69.0,...,48.0,19.85,10.82,30.41,-56.28,-46.34,False,0.00,0.41,0


In [8]:
df.isna().sum()

game_id                    0
period                     0
game_seconds               0
x                          0
y                          0
shot_type                  0
empty_net                  0
strength              269813
prev_type                  0
prev_x                     0
prev_y                     0
time_since_prev            0
distance_from_prev         0
distance_goal              0
prev_distance_goal         0
angle_shot                 0
prev_angle_shot            0
bounce                     0
angle_change               0
speed                      0
is_goal                    0
dtype: int64

In [11]:
df.dtypes

game_id                 int64
period                  int64
game_seconds            int64
x                     float64
y                     float64
shot_type              object
empty_net                bool
strength               object
prev_type              object
prev_x                float64
prev_y                float64
time_since_prev       float64
distance_from_prev    float64
distance_goal         float64
prev_distance_goal    float64
angle_shot            float64
prev_angle_shot       float64
bounce                   bool
angle_change          float64
speed                 float64
is_goal                 int64
dtype: object

In [14]:
df2 = w.encodeCategories(df, ['shot_type', 'strength', 'prev_type'])


In [19]:
newnames = {'shot_type_Slap Shot':'shot_type_Slap_Shot', 'shot_type_Snap Shot':'shot_type_Snap_Shot', 'shot_type_Wrist Shot':'shot_type_Wrist_Shot', 'strength_Power Play':'strength_Power_Play', 'strength_Short Handed':'strength_Short_Handed'}
df2= df2.rename(columns=newnames)

In [20]:
df2.dtypes

game_id                    int64
period                     int64
game_seconds               int64
x                        float64
y                        float64
empty_net                   bool
prev_x                   float64
prev_y                   float64
time_since_prev          float64
distance_from_prev       float64
distance_goal            float64
prev_distance_goal       float64
angle_shot               float64
prev_angle_shot          float64
bounce                      bool
angle_change             float64
speed                    float64
is_goal                    int64
shot_type_Deflected         bool
shot_type_Slap_Shot         bool
shot_type_Snap_Shot         bool
shot_type_Tip-In            bool
shot_type_Wrap-around       bool
shot_type_Wrist_Shot        bool
strength_Power_Play         bool
strength_Short_Handed       bool
prev_type_FACEOFF           bool
prev_type_GIVEAWAY          bool
prev_type_GOAL              bool
prev_type_HIT               bool
prev_type_

In [21]:
df2.to_pickle('./TrainValSets2.pkl')

## Log data

We log game "Winnipeg vs Washington" the 12th march 2018 (game 2017021065) into Comet.ml

In [ ]:
game_df = df[df.game_id == 2017021065]
game_df.head()

In [ ]:
import os
from comet_ml import Experiment

exp = Experiment(
        api_key=os.environ.get('COMET_API_KEY'),
        workspace='ift6758-a5-nhl',
        project_name='milestone2'
    )

exp.log_dataframe_profile(
    game_df,
    name='wpg_v_wsh_2017021065',
    dataframe_format='csv',
    )

exp.end()